In [31]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVR, SVC

from sklearn.metrics import mean_squared_error, f1_score

In [32]:
# Генерируем уникальный seed
my_code = "Fakhrutdinov"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [33]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [34]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [35]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [36]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [37]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [38]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [39]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [40]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,Species
0,0.14847,0.357282,0.356364,0.347973,0.353598,0.444942,Perch
1,0.969588,0.941748,0.938182,0.932432,0.456891,0.718369,Pike
2,0.072441,0.223301,0.229091,0.231419,0.230088,0.369291,Perch
3,0.020741,0.104854,0.103636,0.125,0.140394,0.172023,Roach
4,0.099811,0.223301,0.223636,0.243243,0.445051,0.333051,Parkki
...,...,...,...,...,...,...,...
90,0.084606,0.256311,0.26,0.260135,0.24522,0.364005,Perch
91,0.495165,0.565049,0.556364,0.548986,0.621229,0.88856,Perch
92,0.413053,0.464078,0.465455,0.513514,0.827995,0.609326,Bream
93,0.0694,0.215534,0.210909,0.226351,0.260474,0.35614,Roach


In [41]:
# Задание №1 - анализ метода опорных векторов в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [42]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Length2', 'Length3', 'Length1']
Height


In [43]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [44]:
x_train

,Length2,Length3,Length1
0,0.356364,0.347973,0.357282
1,0.938182,0.932432,0.941748
2,0.229091,0.231419,0.223301
3,0.103636,0.125,0.104854
4,0.223636,0.243243,0.223301
...,...,...,...
90,0.26,0.260135,0.256311
91,0.556364,0.548986,0.565049
92,0.465455,0.513514,0.464078
93,0.210909,0.226351,0.215534


In [45]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
r_model = SVR()

In [46]:
 r_model_1 = SVR(kernel='linear', C=0.8)
r_model_2 = SVR(kernel='poly', C=0.9)
r_model_3 = SVR(kernel='rbf', C=1.0)
r_model_4 = SVR(kernel='sigmoid', C=0.7)

r_models = []
r_models.append(r_model_1)
r_models.append(r_model_2)
r_models.append(r_model_3)
r_models.append(r_model_4)

for model in r_models:
    model.fit(x_train, y_train)
    
mse_list = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mse_list.append(mse)
    print(mse)

0.019350557367850234
0.018541459859700307
0.010276690468768623
13.669313797670007


In [47]:
print("Модель с минимальной среднеквадратичной MSE: ", mse_list.index(min(mse_list))+1)

Модель с минимальной среднеквадратичной MSE:  3


In [48]:
i_min = mse_list.index(min(mse_list))
r_model = r_models[i_min]
r_model.get_params()
      
test_pred = r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.026080076507997436


In [49]:
# Задание №2 - анализ метода опорных векторов в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [50]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Length1', 'Height']
Species


In [51]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [52]:
x_train

,Length1,Height
0,0.357282,0.353598
1,0.941748,0.456891
2,0.223301,0.230088
3,0.104854,0.140394
4,0.223301,0.445051
...,...,...
90,0.256311,0.24522
91,0.565049,0.621229
92,0.464078,0.827995
93,0.215534,0.260474


In [53]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
c_model = SVC()

In [54]:
c_model_1 = SVC(kernel='linear', C=0.8)
c_model_2 = SVC(kernel='poly', C=0.9)
c_model_3 = SVC(kernel='rbf', C=1.0)
c_model_4 = SVC(kernel='sigmoid', C=0.7)

In [55]:
c_models = []
c_models.append(c_model_1)
c_models.append(c_model_2)
c_models.append(c_model_3)
c_models.append(c_model_4)

In [56]:
for model in c_models:
    model.fit(x_train, y_train)

In [57]:
f1_list = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1_list.append(f1)
    print(f1)

0.4910714285714286
0.6917892156862745
0.6645833333333333
0.17578125000000003


In [58]:
print('Модель с максимальным значением F1: ', f1_list.index(max(f1_list))+1)

Модель с максимальным значением F1:  2


In [59]:
i_min = f1_list.index(max(f1_list))
c_model = c_models[i_min]
c_model.get_params()

{'C': 0.9,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'poly',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [60]:
test_pred = c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.759375
